# **Testing Trained PBC4cip**

This document will explore the testing of individual utterances to dialogue prompts found in the EmpatheticConversations database using the PBC4cip classification algorithm. In order to test the utterances please see further into the document for the random prompt and input a response as you see feet. Remember to keep the following files in the environment:

1.   EmpatheticConversations.xlsx - to obtain the prompts
2.   Empathyabase.csv - to ensure the format of the utterance is correct
3.   trained_pbc4cip.sav - trained PBC4cip classifier on the Empathyabase.csv database.





In [200]:
pip install paralleldots

Note: you may need to restart the kernel to use updated packages.


In [201]:
pip install PBC4cip

Note: you may need to restart the kernel to use updated packages.


In [202]:
#Pandas and numpy imports
import pandas as pd
import numpy as np
from pandas import read_csv

#PBC4cip import
#from PBC4cip import PBC4cip
import os
import argparse
import numpy as np
import pandas as pd

from tqdm import tqdm, trange
from PBC4cip import PBC4cip
from PBC4cip.core.Evaluation import obtainAUCMulticlass
from PBC4cip.core.Helpers import get_col_dist, get_idx_val

#Pickle import
import pickle

#Import paralleldots
import paralleldots
from paralleldots import taxonomy
from paralleldots import set_api_key, get_api_key

#Import extras

import requests
import json
import re
import random

# Setup

In [203]:
df = pd.read_excel('EmpatheticConversations.xlsx')
df_prepared = pd.read_csv('Empathyabase.csv')
prompt_df = df[df['utterance_idx'] == 1]
prompt_df = prompt_df.reset_index()

### Paralleldots licence

This license is the one used by the creators of this document, the availability of the API might be inconsistent as a result. If you want to make sure you have access, please sign up through paralleldots and use an available license.

In [204]:
paralleldots.set_api_key('9x4Ya0ooRZDwypZZzsIXOaNywIM6szzkk6yGZMX8e2U')
print( "API Key: %s" % paralleldots.get_api_key() )

API Key: 9x4Ya0ooRZDwypZZzsIXOaNywIM6szzkk6yGZMX8e2U


### Paralleldots

Definition of functions that get the paralleldots data

In [205]:
api_key  = get_api_key()

In [206]:
#Sentiment analysis
def get_sentiment_pd(prompt_values):
    prompt_values['Sentiment'] = str(paralleldots.sentiment(response))
    return prompt_values
#Emotion analysis
def get_emotion_pd(prompt_values, api_key):
    while("Error" in str(prompt_values['Emotion'])): 
        prompt_values['Emotion'] = requests.post( "https://apis.paralleldots.com/v4/emotion", data= { "api_key": api_key, "text": response, "lang_code": 'en' } ).text
        re_emo = re.sub(r"\"", "'", prompt_values['Emotion'])
        prompt_values['Emotion'] = re_emo
        if "error-details" in str(prompt_values['Emotion']):
          print("Error contacting paralleldots!!!!")
          print("Trying again... ")
          prompt_values['Emotion'] = 'Error'
        else:
          prompt_values['Emotion'] = re_emo
    return prompt_values
#Taxonomic analysis
def get_Taxonomy_pd(prompt_values,api_key):
    while('Error' in str(prompt_values['Taxonomy'])):
        tax = requests.post( "https://apis.paralleldots.com/v4/taxonomy", data= { "api_key": api_key, "text": response } ).text
        re_tax = re.sub(r"\"", "'", tax)
        prompt_values['Taxonomy'] = re_tax

        if "error-details" in str(prompt_values['Taxonomy']):
            print("Error contacting paralleldots!!!!")
            print("Trying again... ")
            prompt_values['Taxonomy'] = 'Error'
        else:
            prompt_values['Taxonomy'] = re_tax
    return prompt_values
#Intent analysis
def get_Intent_pd(prompt_values):
    prompt_values['Intent'] = str(paralleldots.intent(str(response)))
    return prompt_values

### Utterance processing

In this part, we define the functions for utterance processing

In [207]:
#Function definitions
def sentiment_separator(s,n):
  s2 = s[s.find(":")+1+s.find('{'):s.find("}")]
  s3 = s2[s2.find('{')+1:]
  array = s3.split(", ")
  #print(array)
  return float(array[n][array[n].find(':')+1:])

intents = ['news','query','spam','marketing','feedback','complaint','suggestion','appreciation']
num_to_context = {0: 'afraid', 1: 'angry', 2: 'annoyed',3: 'anticipating',4: 'anxious',5: 'apprehensive',6: 'ashamed',
                  7: 'caring',8: 'confident',9: 'content', 10: 'devastated', 11: 'disappointed', 12: 'disgusted',
                  13: 'embarrassed', 14: 'excited', 15: 'faithful', 16: 'furious', 17: 'grateful', 18: 'guilty', 
                  19: 'hopeful', 20: 'impressed', 21: 'jealous', 22: 'joyful', 23: 'lonely', 24: 'nostalgic', 
                  25: 'prepared', 26: 'proud', 27: 'sad', 28: 'sentimental', 29: 'surprised', 30: 'terrified', 
                  31: 'trusting'}
unique_tax_array = ['IMPACT', 'EDUCATION', 'POLITICS', 'ENTERTAINMENT', 'TASTE',
                    'SPORTS', 'HEALTHYLIVING', 'GREEN', 'BUSINESS', 'WORLDPOST',
                    'TECH', 'SCIENCE', 'ARTS&CULTURE', 'CRIME', 'TRAVEL', 'RELIGION']
def get_intent(s):
  s = s[s.find("':")+4:]
  arr = s.split(',')
  arr
  if len(arr) > 5:
    s = ''+ arr[4][:arr[4].find('{')]+ arr[4][arr[4].find("re':")+4:]
    arr[4] = s
    k = arr[5]
    k = k[k.find(': {')+3:]
    arr[5] = k
    t = arr[7]
    t = t[:t.find("}}")]
    arr[7] = t
  else:
    arr[4] = arr[4][:arr[4].find("}}")]
    arr.append("'complaint': 0.0")
    arr.append("'suggestion': 0.0")
    arr.append("'appreciation': 0.0")
  for i in range(len(arr)):
    arr[i] = arr[i].replace("'", "")
    arr[i] = arr[i].replace(" ", "")
    val = arr[i].split(':')
    arr[i] = [val[0],float(val[1])]
  return arr

def obtain_tax_sc(s):
#s = df['Taxonomy'][0]
  s = s[s.find("[")+1:]
  s = s[s.find("{")+1:s.find("}]}")]
  s = s.split('},{')
  for i in range(len(s)):
    arr = s[i].split(',')
    new_arr = [arr[1][arr[1].find(":'")+2:len(arr[1])-1],float(arr[0][arr[0].find(':')+1:])]
    #print(new_arr)
    s[i] = new_arr
  return s

def obtain_tax(s):
  s = s[s.find("[")+1:]
  s = s.split(',')
  #print(s)
  for x in s:
    if 'tag' not in s:
      s.remove(x)
  for i in range(len(s)):
    s[i] = s[i][s[i].find(':')+2:s[i].find("}")-1]
  return s


def process_utterance(prompt_values):
    #Sentiment separation
    prompt_values['Negative_score'] = sentiment_separator(prompt_values['Sentiment'],0)
    prompt_values['Neutral_score'] = sentiment_separator(prompt_values['Sentiment'],1)
    prompt_values['Positivity_score'] = sentiment_separator(prompt_values['Sentiment'],2)
    #Context encoding
    context_dict = dict((v, k) for k, v in num_to_context.items())
    prompt_values['context_encoded'] = context_dict[prompt_values['context']]
    #Emotion separation
    emo_str = str(prompt_values['Emotion'])
    emo_str = emo_str[emo_str.find(':{')+3:emo_str.find('}}')]
    emo_arr = emo_str.split(',')
    emo_dic = {}
    for x in emo_arr:
      val = x.split(":")
      val[0] = re.sub(r"\'", "", val[0])
      emo_dic.update({re.match(r'[A-Za-z]*',val[0])[0]:float(val[1])})
    for emo in emo_dic:
      prompt_values[emo] = emo_dic[emo]
    #Intent separation
    intent_array = get_intent(prompt_values['Intent'])
    for x in intent_array:
      prompt_values[str(x[0])] = x[1]
    #Taxonomy separation
    tax_dict = {}
    tax_array = obtain_tax_sc(prompt_values['Taxonomy'].strip())
    for tax in tax_array:
      tax_dict.update({tax[0]:tax[1]})
    tax_dict
    for tax in unique_tax_array:
      if tax in tax_dict:
        prompt_values[tax] = tax_dict[tax]
      else:
        prompt_values[tax] = float(0)
    return prompt_values

In [208]:
def pop_features(prompt_values):
    #Getting rid of unnecessary features
    prompt_values.pop('context')
    prompt_values.pop('conv_id')
    prompt_values.pop('utterance')
    prompt_values.pop('prompt')
    prompt_values.pop('Sentiment')
    prompt_values.pop('Emotion')
    prompt_values.pop('Intent')
    prompt_values.pop('speaker_idx')
    prompt_values.pop('Taxonomy')
    return prompt_values

In [209]:
#Initializing values
def initialize_value(prompt_values, utterance,prompt,df):
    for c in df.columns:
      prompt_values.update({c:prompt[c]})
    prompt_values['utterance_idx'] = 2
    prompt_values['speaker_idx'] = prompt_values['speaker_idx']+1
    prompt_values['utterance'] = utterance
    prompt_values['ut_len'] = len(utterance)
    prompt_values['Talker'] = 2
    prompt_values['Emotion'] = 'Error'
    prompt_values['Taxonomy'] = 'Error'
    return prompt_values

In [210]:
def process_utterance_full(utt, prompt_values, dataframe, api, promp):
    prompt_values = initialize_value(prompt_values, utt, promp, dataframe)
    prompt_values = get_sentiment_pd(prompt_values)
    prompt_values = get_emotion_pd(prompt_values, api)
    prompt_values = get_Taxonomy_pd(prompt_values, api)
    prompt_values = get_Intent_pd(prompt_values)
    prompt_values = process_utterance(prompt_values)
    prompt_values = pop_features(prompt_values)
    return prompt_values

### Classification

In [278]:
def to_dataframe_format(df_to_copy, utterance):
    df_prepared2 = df_to_copy.copy()
    df_prepared2.loc[len(df_prepared2)] = utterance
    test = pd.DataFrame(df_prepared2, index=[len(df_prepared2)-1])
    test['Empathy'] = test['Empathy'].astype('string')
    test["utterance_idx"] = test["utterance_idx"].astype('category')
    test["Talker"] = test["Talker"].astype('category')
    test["context_encoded"] = test["context_encoded"].astype('category')
    return test

In [283]:
def obtain_predictions(model_filename,dataframe):
    X1_test = dataframe.drop(columns=['Empathy'])
    Y1_test = dataframe.drop(columns=X1_test.columns)
    #filename = 'trained_pbc4cip.sav'
    # load the model from disk
    loaded_model = pickle.load(open(model_filename, 'rb'))
    y_pred = loaded_model.predict(X1_test)
    return y_pred

# Interaction

## Get prompt

In [268]:
number_of_prompt = random.randint(0,len(prompt_df))
#number_of_prompt = 27
prompt = prompt_df.iloc[number_of_prompt]
print("Your conversation partner says: ")
print(prompt['utterance'])

Your conversation partner says: 
I love volunteering.


## Get response

In [290]:
response = input()

I love it very much too! 


In [291]:
utterance_values = {}
utterance_values = process_utterance_full(response,utterance_values,df,api_key, prompt)

{'utterance_idx': 2,
 'ut_len': 25,
 'Empathy': 3,
 'Talker': 2,
 'Negative_score': 0.01,
 'Neutral_score': 0.128,
 'Positivity_score': 0.862,
 'context_encoded': 7,
 'Sad': 0.0526198349,
 'Excited': 0.2249661564,
 'Angry': 0.0345385505,
 'Fear': 0.1638114188,
 'Bored': 0.0,
 'Happy': 0.5240640393,
 'news': 0.001,
 'query': 0.032,
 'spam': 0.195,
 'marketing': 0.013,
 'feedback': 0.759,
 'complaint': 0.002,
 'suggestion': 0.003,
 'appreciation': 0.995,
 'IMPACT': 0.1361103207,
 'EDUCATION': 0.0,
 'POLITICS': 0.0,
 'ENTERTAINMENT': 0.5995883346,
 'TASTE': 0.0,
 'SPORTS': 0.0,
 'HEALTHYLIVING': 0.0,
 'GREEN': 0.0,
 'BUSINESS': 0.0,
 'WORLDPOST': 0.0,
 'TECH': 0.1640076041,
 'SCIENCE': 0.0,
 'ARTS&CULTURE': 0.0,
 'CRIME': 0.0,
 'TRAVEL': 0.0,
 'RELIGION': 0.0}

## Classification

In this section, we carry out classification using the saved PBC4cip model

In [293]:
ut_to_df =  to_dataframe_format(df_prepared,utterance_values)
#print(ut_to_df)
filename = 'trained_pbc4cip.sav'
prediction = obtain_predictions(filename,ut_to_df)
print("The classifier predicts that your response presents an empathy level of: "+ str(prediction[-1]) +" on a scale from 1 to 5")

The classifier predicts that your response presents an empathy level of: 4 on a scale from 1 to 5


# Examples

We include some examples of exchange pairs

In [298]:
prompt = prompt_df.iloc[27]
print("Your conversation partner says: ")
print(prompt['utterance'])
response = "I love it very much too!"
print("You say:")
print(response)

utterance_values = {}
utterance_values = process_utterance_full(response,utterance_values,df,api_key, prompt)
ut_to_df =  to_dataframe_format(df_prepared,utterance_values)
filename = 'trained_pbc4cip.sav'
prediction = obtain_predictions(filename,ut_to_df)

print("The classifier predicts that your response presents an empathy level of: "+ str(prediction[-1]) +" on a scale from 1 to 5")

Your conversation partner says: 
I love volunteering.
You say:
I love it very much too!


The classifier predicts that your response presents an empathy level of: 4 on a scale from 1 to 5


In [304]:
prompt = prompt_df.iloc[303]
print("Your conversation partner says: ")
print(prompt['utterance'])
response = "I am so glad you didn't win "
print("You say:")
print(response)

utterance_values = {}
utterance_values = process_utterance_full(response,utterance_values,df,api_key, prompt)
ut_to_df =  to_dataframe_format(df_prepared,utterance_values)
filename = 'trained_pbc4cip.sav'
prediction = obtain_predictions(filename,ut_to_df)

print("The classifier predicts that your response presents an empathy level of: "+ str(prediction[-1]) +" on a scale from 1 to 5")

Your conversation partner says: 
Last week a friend of mine won $2000 on a scratch off ticket. I buy those things all the time and never win squat.
You say:
I am so glad you didn't win 


The classifier predicts that your response presents an empathy level of: 1 on a scale from 1 to 5


In [309]:
prompt = prompt_df.iloc[71]
print("Your conversation partner says: ")
print(prompt['utterance'])
response = "Really? I feel fantastic!"
print("You say:")
print(response)

utterance_values = {}
utterance_values = process_utterance_full(response,utterance_values,df,api_key, prompt)
ut_to_df =  to_dataframe_format(df_prepared,utterance_values)
filename = 'trained_pbc4cip.sav'
prediction = obtain_predictions(filename,ut_to_df)

print("The classifier predicts that your response presents an empathy level of: "+ str(prediction[-1]) +" on a scale from 1 to 5")

Your conversation partner says: 
Man.....my cat died:( I feel horrible.
You say:
Really? I feel fantastic!


The classifier predicts that your response presents an empathy level of: 2 on a scale from 1 to 5


In [311]:
prompt = prompt_df.iloc[124]
print("Your conversation partner says: ")
print(prompt['utterance'])
response = "I don't really care"
print("You say:")
print(response)

utterance_values = {}
utterance_values = process_utterance_full(response,utterance_values,df,api_key, prompt)
ut_to_df =  to_dataframe_format(df_prepared,utterance_values)
filename = 'trained_pbc4cip.sav'
prediction = obtain_predictions(filename,ut_to_df)

print("The classifier predicts that your response presents an empathy level of: "+ str(prediction[-1]) +" on a scale from 1 to 5")

Your conversation partner says: 
oh boy....I just screwed my whole life up and I want to die
You say:
I don't really care


The classifier predicts that your response presents an empathy level of: 1 on a scale from 1 to 5
